https://tianchi.aliyun.com/competition/entrance/231593/information

In [1]:
import pandas as pd
import numpy as np

In [2]:
dfoff = pd.read_csv('./data/ccf_offline_stage1_train.csv')
dftest = pd.read_csv('./data/ccf_offline_stage1_test_revised.csv')
dfon = pd.read_csv('./data/ccf_online_stage1_train.csv')

dfsub = pd.DataFrame({'User_id':'用户ID',
                      'Coupon_id':'优惠券ID',
                      'Date_received':'领取优惠券日期',
                      'Probability':'预测值'},index=[1])

In [4]:
for i in [dfoff,dfon,dftest]:
    print("=="*50)
    print(i.head(2))

   User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0  1439408         2632        NaN           NaN       0.0            NaN   
1  1439408         4663    11002.0        150:20       1.0     20160528.0   

         Date  
0  20160217.0  
1         NaN  
    User_id  Merchant_id  Action  Coupon_id Discount_rate  Date_received  \
0  13740231        18907       2  100017492        500:50     20160513.0   
1  13740231        34805       1        NaN           NaN            NaN   

         Date  
0         NaN  
1  20160321.0  
   User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received
0  4129537          450       9983          30:5       1.0       20160712
1  6949378         1300       3429          30:5       NaN       20160706


# 理解一下题意

测试集(dftest)列如下，意思是，用户已经，在某个日期，拿到了商家的某个优惠券（线上还是线下不清楚，但是这里可以理解为，拿和用没有细分了）  
提交结果(dfsub)如下，预测上表中的用户，在领取优惠券15天内，使用优惠券的概率

In [4]:
dfsub.head(1)

,User_id,Coupon_id,Date_received,Probability
1,用户ID,优惠券ID,领取优惠券日期,预测值


In [5]:
dftest.head(1)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,20160712


# AUC到底怎么算

1、每个优惠券ID都算一次，最后取平均  
2、样本全为0，pass,也不计入最后的平均  
3、见evaluation  

# 理解一下数据集

要么拿了券，要么有购买行为。没有不拿券也不消费的。

线下没有fixed,也就是打折行为，营销方式只有优惠券一种

无论线上还是线下，拿券消费是最少的

线上，1%打折下单、1%优惠券下单，92%直接消费，6%拿券但是不消费。  
也就是说线上拿券的人占7%，但是线上核销的只有1%  

线下，很明显就是官方说的偏采样了，60%的人拿了券，但是只有4%核销，实际上拿券消费的人必然是过采样的。

In [6]:
dfon_notfix = dfon[dfon.Coupon_id!='fixed']
dfon_isfix = dfon[dfon.Coupon_id=='fixed']

def get_cd(dfoff):
    # 拿券，消费
    cd1 = dfoff[(dfoff.Coupon_id.notna()) & (dfoff.Date.notna())].User_id.count()
    # 拿券，不消费
    cd2 = dfoff[(dfoff.Coupon_id.notna()) & (dfoff.Date.isna())].User_id.count()
    #不拿券，消费
    cd3 = dfoff[(dfoff.Coupon_id.isna()) & (dfoff.Date.notna())].User_id.count()
    #不拿券，不消费
    cd4 = dfoff[(dfoff.Coupon_id.isna()) & (dfoff.Date.isna())].User_id.count()
    
    for a,b in zip(['优惠券消费','直接消费','拿券不消费','不拿券也不消费'],[cd1,cd3,cd2,cd4]):
        c = np.around(b/sum([cd1,cd2,cd3])*100)
        print('{0:7}\t{1:10}\t{2:6}%'.format(a,b,c))
        #print(a, b, )

In [7]:
for i in zip([dfon,dfon_notfix,dfon_isfix,dfoff],['线上','线上无fix','线上且fix','线下']):
    print('=='*7+i[1]+'=='*20)
    print(get_cd(i[0]),'\n')

==============线上========================================
优惠券消费  	    216459	   2.0%
直接消费   	  10557469	  92.0%
拿券不消费  	    655898	   6.0%
不拿券也不消费	         0	   0.0%
None 

==============线上无fix========================================
优惠券消费  	     84913	   1.0%
直接消费   	  10557469	  93.0%
拿券不消费  	    655898	   6.0%
不拿券也不消费	         0	   0.0%
None 

==============线上且fix========================================
优惠券消费  	    131546	 100.0%
直接消费   	         0	   0.0%
拿券不消费  	         0	   0.0%
不拿券也不消费	         0	   0.0%
None 

==============线下========================================
优惠券消费  	     75382	   4.0%
直接消费   	    701602	  40.0%
拿券不消费  	    977900	  56.0%
不拿券也不消费	         0	   0.0%
None 



# 一些比较迷惑的地方：

根据题意，线上只是作为背景参考使用，它不构成正负样本，所以线上样本的优惠券，不参与线下的活动？【很可能不参与】  
线上的，优惠券是null但是消费了（直接消费）,一定是没有优惠券吗？会不会是商家没有发券呢？还是重复消费没有那么多优惠券呢?【假设一】    
线下的，所有样本全部都是“有券可以拿”的吗？【同假设一，但是测试集一定是全有券】   
o2o,线上到线下，意思是线上取优惠券，可以在线上或者线下核销,dfon的优惠券消费是全都是线下吗？【倾向于优惠券消费是线上的】  
线下，没有fixed行为吗？【假设二】

【假设一】：所有消费，哪怕没有领取，消费券也是默认用，除非没有券。也就是所有的“普通消费”，全都默认商家没有发券，也没有打折。  
【假设二】：所有线下营销只有优惠券，没有打折行为  


# 关于dfon的Action字段，当等于
0: 点击，优惠券为null,Date全有 | 0代表没有拿优惠券，但是消费了？|就是直接消费  
1：购买, 优惠券三种都有[null,id,fixed],Date全有 | 1代表肯定购买了，但是不一定用了消费券，    
2: 领取优惠券，优惠券为id,Date全无 | 2代表只拿了优惠券，但是没有交易  

在0和1之间，都存在优惠券为null,但是消费日期非空的样本。看了数据后并没有发现有何区别。后续可以进一步试试。

In [10]:
# dfth = dfon[(dfon.Date.notna()) & (dfon.Coupon_id.isna())]

# 关于两个训练集之间用户id、商家ID、优惠券id的交叉情况

线上下训练集是有交叉的，商家和优惠券都没有交叉！

用户： 线上4~15500000，线下4~7361032,有交集267488个  
商家： 线上10001+，线下1+，无交集  
优惠券:线上100000003~100233610，线下1~14045,无交集  

另外，说每个优惠券id只对应一个商家id

In [64]:
def get_jbb(s1,s2):
    print(len(s1),len(s2),len(set(s1).intersection(s2)))
    
id_list = ['User_id','Merchant_id','Coupon_id']
lon = []
loff = []
for i in id_list:
    lon.append(dfon[i].unique())
    loff.append(dfoff[i].unique())
for i,j in zip([0,1,2],id_list):
    print('\n'+j)
    get_jbb(lon[i],loff[i])


User_id
762858 539438 267448

Merchant_id
7999 8415 0

Coupon_id
27748 9739 0


# 初步的构思

1、线上数据集

注意618的影响
领取优惠券的时间接近周末？  
领取消费券的时间接近？   
领取优惠券的对应商家距离近？  
领取优惠券的力度大？（门槛低，金额高）  
领取优惠券的时间越近消费的概率越高？  
领取消费券的时间，靠近放假时间？  
领取优惠券越多，核销的概率越大？  
不同折扣，不同优惠方式 ，哪种核销概率高？
领取到核销的时间特征  
领券时间和用券消费的时间间隔有什么特征，为什么预测的是“15天”

人本身消费属性？（RFM，结合线上数据来定义消费者属性）  
人和固定商家有消费关系？(喜欢这个商家)  
线下偏好、线上偏好  
被营销偏好  
优惠券id,不同人，有一样的吗？

商品本身费用高？  
商品本身消费的人群就少？  
商家品质不咋地，或者本身不善于这种营销？（过往成绩就不好）  


线上优惠券消费，和线下优惠券核销，可能不是一种消费习惯，需要鉴别